In [ ]:
import numpy as np

class Activation:
    @staticmethod
    def sigmoid(x):
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def sigmoid_derivative(x):
        sigmoid_x = Activation.sigmoid(x)
        return sigmoid_x * (1 - sigmoid_x)

    @staticmethod
    def relu(x):
        return np.maximum(0, x)

    @staticmethod
    def relu_derivative(x):
        return np.where(x > 0, 1, 0)

In [ ]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size, activation=None, init_method="random", weights=None, biases=None):
        self.input_size = input_size
        self.output_size = output_size

        if weights is not None and biases is not None:
            self.weights = np.array(weights)
            self.biases = np.array(biases)
        else:
            self.init_weights(input_size, output_size, init_method)
            self.biases = np.zeros((1, output_size))

        # Set activation function
        self.set_activation(activation)

    def init_weights(self, input_size, output_size, method="random"):
        if method == "random":
            self.weights = np.random.randn(input_size, output_size) * 0.1
        elif method == "xavier":
            self.weights = np.random.randn(input_size, output_size) * np.sqrt(1 / input_size)
        elif method == "he":
            self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        else:
            raise ValueError("Unknown initialization method. Use 'random', 'xavier', or 'he'.")

    def set_activation(self, activation):
        activations = {
            "sigmoid": (Activation.sigmoid, Activation.sigmoid_derivative),
            "relu": (Activation.relu, Activation.relu_derivative)
        }
        self.activation, self.activation_derivative = activations.get(activation, (None, None))

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        self.a = self.activation(self.z) if self.activation else self.z
        return self.a

    def backward(self, dA, learning_rate):
        dZ = dA * (self.activation_derivative(self.z) if self.activation_derivative else 1)
        dW = np.dot(self.inputs.T, dZ)
        dB = np.sum(dZ, axis=0, keepdims=True)
        dA_prev = np.dot(dZ, self.weights.T)

        # Update weights and biases
        self.weights -= learning_rate * dW
        self.biases -= learning_rate * dB

        return dA_prev


In [ ]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, input_size, output_size, activation=None, init_method="random", weights=None, biases=None):
        """Add a new layer to the network."""
        self.layers.append(Layer(input_size, output_size, activation, init_method, weights, biases))

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, X, Y, learning_rate):
        output_layer = self.layers[-1]
        dA = (output_layer.a - Y)

        for layer in reversed(self.layers):
            dA = layer.backward(dA, learning_rate)

    def train(self, X, Y, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, Y, learning_rate)
            loss = np.mean((output - Y) ** 2)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """Forward pass to get predictions"""
        output = X
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def summary(self):
        print("\n" + "=" * 40)
        print("Neural Network Summary")
        print("=" * 40)
        print(f"{'Layer':<10}{'Input Shape':<15}{'Output Shape':<15}{'Params'}")
        print("-" * 40)

        total_params = 0
        input_shape = None

        for i, layer in enumerate(self.layers):
            input_shape = (None, layer.input_size) if input_shape is None else (None, self.layers[i-1].output_size)
            output_shape = (None, layer.output_size)

            num_params = (layer.input_size * layer.output_size) + layer.output_size
            total_params += num_params

            print(f"{i+1:<10}{str(input_shape):<15}{str(output_shape):<15}{num_params}")

        print("=" * 40)
        print(f"Total Parameters: {total_params}")
        print("=" * 40)


In [ ]:
def accuracy(y_true, y_pred):
      """Calculate accuracy by comparing true labels with predicted labels."""
      correct = np.sum(y_true == y_pred)
      total = len(y_true)
      return correct / total

In [ ]:
# XOR Data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

# Create Model
nn = NeuralNetwork()
nn.add_layer(2, 4, "sigmoid")
nn.add_layer(4, 1, "sigmoid")

# Train

nn.train(X, Y, epochs=1000, learning_rate=0.01)

# Test
print(nn.forward(X))



Epoch 0, Loss: 0.2500
Epoch 100, Loss: 0.2500
Epoch 200, Loss: 0.2500
Epoch 300, Loss: 0.2500
Epoch 400, Loss: 0.2500
Epoch 500, Loss: 0.2500
Epoch 600, Loss: 0.2500
Epoch 700, Loss: 0.2500
Epoch 800, Loss: 0.2500
Epoch 900, Loss: 0.2500
[[0.49998923]
 [0.50016451]
 [0.49986853]
 [0.50004564]]


In [ ]:
nn.summary()


Neural Network Summary
Layer     Input Shape    Output Shape   Params
----------------------------------------
1         (None, 2)      (None, 4)      12
2         (None, 4)      (None, 1)      5
Total Parameters: 17


In [ ]:
# 1st Question
custom_weights_1 = np.array([[0.15, 0.25], [0.2, 0.3]])
custom_biases_1 = np.array([[0.35, 0.35]])

custom_weights_2 = np.array([[0.4, 0.2], [0.45, 0.55]])
custom_biases_2 = np.array([[0.16, 0.16]])

nn1 = NeuralNetwork()
nn1.add_layer(2, 2, "sigmoid", weights=custom_weights_1, biases=custom_biases_1)
nn1.add_layer(2, 2, "sigmoid", weights=custom_weights_2, biases=custom_biases_2)
XN=np.array([0.05,0.1])
ans=nn1.predict(XN)
print(ans)

[[0.66058583 0.64724255]]


In [ ]:
#2nd Question
import numpy as np

# Define training data
X = np.array([[1, 1], [0, 1]])  # Inputs
Y = np.array([[0], [1]])        # Targets

# Given initial weights and biases
W1 = np.array([[0.5, 0.2], [-0.3, 0.5]])
B1 = np.array([[0.1, 0.3]])

W2 = np.array([[0.6], [-0.4]])
B2 = np.array([[0.8]])

# Create Neural Network
nn2 = NeuralNetwork()

# Add layers with given weights and biases
nn2.add_layer(2, 2, "relu", weights=W1, biases=B1)  # Hidden layer
nn2.add_layer(2, 1, "relu", weights=W2, biases=B2)  # Output layer

# Train for 5 epochs
nn2.train(X, Y, epochs=500, learning_rate=0.5)

# Print updated weights and biases
print("\nUpdated Weights:")
for i, layer in enumerate(nn2.layers):
    print(f"Layer {i+1} Weights:\n", layer.weights)
    print(f"Layer {i+1} Biases:\n", layer.biases)

# Predict on training data (to check accuracy)
train_predictions = nn2.predict(X)
train_predicted_classes = (train_predictions > 0.5).astype(int)

# Compute accuracy
train_accuracy = accuracy(Y, train_predicted_classes)
print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")

# Predict on new test data
X_new = np.array([[1, 0], [0, 0], [1, 1], [0, 1],[0,1],[1,1]])
predictions = nn2.predict(X_new)
predicted_classes = (predictions > 0.5).astype(int)

# Print predictions and accuracy
print("\nPredictions on Test Data:")
print(predicted_classes)


Epoch 0, Loss: 0.3034
Epoch 100, Loss: 0.0000
Epoch 200, Loss: 0.0000
Epoch 300, Loss: 0.0000
Epoch 400, Loss: 0.0000

Updated Weights:
Layer 1 Weights:
 [[ 0.326       1.176439  ]
 [-0.474       0.06403951]]
Layer 1 Biases:
 [[-0.074      -0.13596049]]
Layer 2 Weights:
 [[ 0.513     ]
 [-0.92605023]]
Layer 2 Biases:
 [[1.]]

Training Accuracy: 100.00%

Predictions on Test Data:
[[0]
 [1]
 [0]
 [1]
 [1]
 [0]]
